<a href="https://colab.research.google.com/github/le0-425/3/blob/main/%EB%8F%99%EA%B3%84%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TensorFlow 및 GPU 지원 버전 설치
!pip install tensorflow tensorflow-gpu

In [ ]:
# 필수 라이브러리 임포트
import tensorflow as tf  # 딥러닝 프레임워크
import numpy as np      # 수치 연산용 라이브러리
import os              # 파일 및 디렉토리 처리
from tensorflow.keras.preprocessing import image  # 이미지 전처리 도구
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # 이미지 증강 도구
from google.colab import drive  # 구글 드라이브 연동
import datetime        # 시간 정보 처리
import zipfile         # ZIP 파일 처리
import matplotlib.pyplot as plt  # 그래프 시각화
from google.colab import drive

In [ ]:
# GPU 사용 가능 여부 확인
print("GPU 사용 가능 여부:", tf.config.list_physical_devices('GPU'))

In [ ]:
class AnimalClassifier:
    def __init__(self, model_path=None):
        """
        동물 분류기 클래스 초기화
        Args:
            model_path: 기존에 학습된 모델의 파일 경로
                       None인 경우 새로운 모델 생성
        """
        self.model = self._load_or_create_model(model_path)
        self.image_size = (150, 150)  # 입력 이미지 크기 (너비, 높이)
        self.batch_size = 32          # 한 번에 처리할 이미지 수
        self.target_animal = None     # 현재 분류 대상 동물

    def _load_or_create_model(self, model_path):
        """
        저장된 모델을 불러오거나 새 모델을 생성
        Args:
            model_path: 모델 파일 경로
        Returns:
            로드된 모델 또는 새로 생성된 모델 객체
        """
        if model_path and os.path.exists(model_path):
            print("기존 모델을 불러옵니다.")
            try:
                return tf.keras.models.load_model(model_path)
            except:
                print("이전 형식(.h5)의 모델을 새로운 형식(.keras)으로 변환합니다.")
                old_model = tf.keras.models.load_model(model_path)
                new_path = model_path.replace('.h5', '.keras')
                old_model.save(new_path)
                return old_model
        else:
            print("새로운 모델을 생성합니다.")
            return self._create_new_model()

    def _create_new_model(self):
        """
        새로운 CNN 모델 생성
        Returns:
            컴파일된 CNN 모델
        """
        model = tf.keras.models.Sequential([
            # 입력층과 첫 번째 컨볼루션 블록
            tf.keras.layers.Conv2D(32, (3, 3), activation='relu',
                                 input_shape=(150, 150, 3)),
            tf.keras.layers.MaxPooling2D((2, 2)),

            # 두 번째 컨볼루션 블록
            tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),

            # 세 번째 컨볼루션 블록
            tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),

            # 네 번째 컨볼루션 블록
            tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),

            # 특성 맵을 1차원으로 평탄화
            tf.keras.layers.Flatten(),

            # 완전연결층
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.5),

            # 출력층
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

        model.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        return model

    def train_on_new_data(self, data_dir, target_animal, epochs=10):
        """
        새로운 데이터로 모델 학습
        """
        self.target_animal = target_animal

        datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            validation_split=0.2
        )

        train_generator = datagen.flow_from_directory(
            data_dir,
            target_size=self.image_size,
            batch_size=self.batch_size,
            class_mode='binary',
            subset='training'
        )

        validation_generator = datagen.flow_from_directory(
            data_dir,
            target_size=self.image_size,
            batch_size=self.batch_size,
            class_mode='binary',
            subset='validation'
        )

        # 체크포인트 디렉토리 생성
        checkpoint_dir = "/content/drive/MyDrive/checkpoints"
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        # 체크포인트 설정 (.keras 확장자 사용)
        checkpoint_path = f"{checkpoint_dir}/{self.target_animal}_cp-{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.keras"
        checkpoint = tf.keras.callbacks.ModelCheckpoint(
            checkpoint_path,
            save_best_only=True,
            monitor='val_accuracy'
        )

        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )

        history = self.model.fit(
            train_generator,
            epochs=epochs,
            validation_data=validation_generator,
            callbacks=[checkpoint, early_stopping]
        )

        self._plot_training_history(history)

        return history

    def _plot_training_history(self, history):
        """
        학습 과정을 그래프로 시각화
        """
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], label='Training Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title('Model Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        plt.tight_layout()
        plt.show()

    def predict(self, image_path, target_animal):
        """
        단일 이미지에 대한 예측 수행
        """
        self.target_animal = target_animal

        img = image.load_img(image_path, target_size=self.image_size)
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, 0)
        img_array /= 255.

        prediction = self.model.predict(img_array)
        return prediction[0][0]

    def save_model(self, save_path):
        """
        학습된 모델을 파일로 저장
        """
        if self.target_animal:
            # .h5를 .keras로 변경
            save_path = save_path.replace('.h5', f'_{self.target_animal}.keras')
        self.model.save(save_path)
        print(f"모델이 저장되었습니다: {save_path}")


In [ ]:
def check_image_count(directory):
    """
    지정된 디렉토리의 이미지 파일 정보를 출력
    Args:
        directory: 확인할 디렉토리 경로
    """
    if not os.path.exists(directory):
        print(f"오류: 디렉토리를 찾을 수 없습니다: {directory}")
        return

    # 지원되는 이미지 형식
    image_extensions = ('.jpg', '.jpeg', '.png', '.gif')

    # 모든 파일 목록
    all_files = os.listdir(directory)

    # 이미지 파일만 필터링
    image_files = [f for f in all_files
                  if os.path.isfile(os.path.join(directory, f))
                  and f.lower().endswith(image_extensions)]

    # 결과 출력
    print(f"\n=== 디렉토리 분석 결과: {directory} ===")
    print(f"전체 파일 수: {len(all_files)}")
    print(f"이미지 파일 수: {len(image_files)}")

    # 이미지 형식별 개수
    format_count = {}
    for img in image_files:
        ext = os.path.splitext(img)[1].lower()
        format_count[ext] = format_count.get(ext, 0) + 1

    print("\n이미지 형식별 개수:")
    for ext, count in format_count.items():
        print(f"{ext}: {count}개")


In [ ]:
# ======================================================
# 메인 실행 함수
# ======================================================

def main():
    """
    프로그램의 메인 실행 함수
    기능:
        1. 구글 드라이브 연동
        2. 모델 초기화
        3. 사용자 인터페이스 제공
        4. 작업 선택 및 실행
    """
    # 구글 드라이브를 /content/drive 경로에 마운트
    # 모델과 데이터를 저장하고 불러오기 위해 필요함
    print("구글 드라이브 연동을 시작합니다...")
    drive.mount('/content/drive')

    # 기본 모델 파일의 경로를 지정
    # 이 경로에 있는 모델을 불러오거나, 없으면 새로 생성함
    MODEL_PATH = '/content/drive/MyDrive/animal_classifier.keras'

    # AnimalClassifier 클래스의 인스턴스를 생성
    # 기존 모델이 있으면 로드하고, 없으면 새로운 모델을 생성
    print("동물 이미지 분류기를 초기화합니다...")
    classifier = AnimalClassifier(MODEL_PATH)

    # 프로그램의 메인 루프 시작
    # 사용자가 종료를 선택할 때까지 반복
    while True:
        # 메뉴 인터페이스 출력
        # 구분선과 함께 메뉴 옵션을 표시
        print("\n" + "="*30)
        print("=== 동물 이미지 분류기 ===")
        print("="*30)
        print("1. 새로운 데이터로 학습")
        print("2. 이미지 예측")
        print("3. 모델 저장")
        print("4. 종료")
        print("="*30)

        # 사용자로부터 메뉴 선택 입력 받기
        choice = input("\n원하는 작업을 선택하세요 (1-4): ")

        # 선택된 메뉴에 따른 작업 실행
        if choice == '1':
            # 새로운 데이터로 모델을 학습하는 기능
            print("\n=== 새로운 데이터로 학습을 시작합니다 ===")
            try:
                # 학습에 필요한 정보들을 사용자로부터 입력 받음
                # 1. 데이터 디렉토리 경로
                data_dir = input("학습 데이터 디렉토리 경로를 입력하세요\n"
                               "(예: /content/drive/MyDrive/animal_data): ")

                # 2. 학습할 동물 종류
                target_animal = input("\n학습할 동물 종류를 입력하세요\n"
                                    "(예: 강아지, 고양이): ")

                # 3. 학습 반복 횟수 (에포크)
                epochs = int(input("\n학습 반복 횟수를 입력하세요\n"
                                 "(권장: 10-50): "))

                # 학습 시작 메시지 출력
                print(f"\n{target_animal} 인식을 위한 학습을 시작합니다...")
                print("학습 중입니다. 이 작업은 몇 분 정도 소요될 수 있습니다...")

                # 모델 학습 실행
                # train_on_new_data 메서드를 호출하여 학습 수행
                history = classifier.train_on_new_data(data_dir, target_animal, epochs)
                print("\n학습이 성공적으로 완료되었습니다!")

                # 최종 학습 결과 출력
                # 학습 데이터와 검증 데이터에 대한 정확도를 표시
                final_acc = history.history['accuracy'][-1]
                final_val_acc = history.history['val_accuracy'][-1]
                print(f"\n최종 학습 정확도: {final_acc:.2%}")
                print(f"최종 검증 정확도: {final_val_acc:.2%}")

            except ValueError as e:
                # 잘못된 입력값(예: 에포크 수에 문자열 입력)에 대한 에러 처리
                print(f"\n에러: 잘못된 입력값입니다. {e}")
            except Exception as e:
                # 기타 모든 예외 상황에 대한 에러 처리
                print(f"\n학습 중 에러가 발생했습니다: {e}")
                print("데이터 경로와 형식을 확인해주세요.")

        elif choice == '2':
            # 이미지 예측 기능
            print("\n=== 이미지 예측을 시작합니다 ===")
            try:
                # 예측할 이미지 경로와 동물 종류 입력 받기
                image_path = input("예측할 이미지 파일의 경로를 입력하세요\n"
                                 "(예: /content/drive/MyDrive/test.jpg): ")
                target_animal = input("\n확인할 동물 종류를 입력하세요\n"
                                    "(예: 강아지, 고양이): ")

                # 입력받은 이미지 파일이 실제로 존재하는지 확인
                if not os.path.exists(image_path):
                    raise FileNotFoundError("입력한 이미지 파일을 찾을 수 없습니다.")

                # 예측 수행
                print("\n이미지를 분석중입니다...")
                prediction = classifier.predict(image_path, target_animal)

                # 예측 결과 출력
                # 0.5를 기준으로 판단 (0.5 초과면 해당 동물, 이하면 아님)
                print("\n=== 예측 결과 ===")
                if prediction > 0.5:
                    print(f"이 이미지는 {target_animal}입니다.")
                    print(f"확률: {prediction:.2%}")
                else:
                    print(f"이 이미지는 {target_animal}가 아닙니다.")
                    print(f"확률: {(1-prediction):.2%}")

            except FileNotFoundError as e:
                # 파일이 존재하지 않는 경우의 에러 처리
                print(f"\n에러: {e}")
                print("이미지 파일의 경로를 정확히 입력해주세요.")
            except Exception as e:
                # 기타 예외 상황에 대한 에러 처리
                print(f"\n예측 중 에러가 발생했습니다: {e}")
                print("이미지 파일 형식을 확인해주세요.")

        elif choice == '3':
            # 모델 저장 기능
            print("\n=== 모델 저장을 시작합니다 ===")
            try:
                # 동물 종류가 설정되지 않은 경우 입력 받기
                if not classifier.target_animal:
                    target_animal = input("저장할 모델의 동물 종류를 입력하세요: ")
                    classifier.target_animal = target_animal

                # 모델을 저장할 디렉토리 생성
                # 없는 경우 새로 만듦
                save_dir = '/content/drive/MyDrive/animal_models'
                if not os.path.exists(save_dir):
                    os.makedirs(save_dir)

                # 현재 시간을 포함한 파일명 생성
                # 중복을 피하기 위해 타임스탬프 사용
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                save_path = f'{save_dir}/animal_classifier_{timestamp}.keras'

                # 모델 저장 실행
                classifier.save_model(save_path)
                print("\n모델이 성공적으로 저장되었습니다!")
                print(f"저장 위치: {save_path}")

            except Exception as e:
                # 모델 저장 중 발생하는 에러 처리
                print(f"\n모델 저장 중 에러가 발생했습니다: {e}")
                print("저장 경로와 권한을 확인해주세요.")

        elif choice == '4':
            # 프로그램 종료
            print("\n프로그램을 종료합니다.")
            print("이용해 주셔서 감사합니다!")
            break

        else:
            # 잘못된 메뉴 선택에 대한 처리
            print("\n잘못된 선택입니다. 1-4 사이의 숫자를 입력해주세요.")

# ======================================================
# 프로그램 시작점
# ======================================================

if __name__ == "__main__":
    try:
        # 프로그램 시작
        print("동물 이미지 분류 프로그램을 시작합니다...")
        main()
    except KeyboardInterrupt:
        # Ctrl+C 등으로 프로그램이 중단된 경우
        print("\n\n프로그램이 사용자에 의해 중단되었습니다.")
    except Exception as e:
        # 예상치 못한 에러가 발생한 경우
        print(f"\n\n예기치 않은 오류가 발생했습니다: {e}")
    finally:
        # 프로그램 종료 시 항상 실행되는 코드
        print("\n프로그램을 종료합니다. 이용해 주셔서 감사합니다!")